<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 · Análisis de sentimientos y textos</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia 
y Telecomunicación</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Módulo 4: Implementación del Deep Learning en el Procesamiento del Lenguaje Natural

## Scripts de los métodos de modelo de lenguaje explicados en el módulo

En este notebook ilustraremos la aplicación de un modelo BERT para hacer dos tareas.<br><br>
La primera tarea es la que hacen los estudiantes de idiomas, conocida como <i>fill in the blanks</i>. Ante un texto con una palabra borrada, la tarea es predecir la palabra que debería estar. Este ejercicio sirve para evaluar un modelo del lenguaje, puesto que un buen modelo del lenguaje es aquél que permite predecir la palabra más adecuada a un contexto.<br>
Este notebook está inspirado en el notebook <a href="https://github.com/ramsrigouthamg/BERT_generate_grammar_MCQ_from_news_article/blob/master/bert-english-vocabulary-mcq-questions.ipynb"><i>bert-english-vocabulary-mcq-questions</i></a>, de ramsrigouthamg<br><br>
La segunda tarea consiste en calcular la similitud entre frases que tienen una palabra polisémica. Concretamente, se demostrará que una frase con el nombre de la compañía <i>Apple</i>, está más cerca de una frase sobre <i>IPhones</i> y <i>acciones</i>, que de una frase que habla sobre pasteles de manzana

## Fill in the blanks

### Instalar el paquete <i>pytorch-pretrained-bert</i> y cargarlo

In [ ]:
installing -> conda install nb_conda
installing -> conda update conda
installing conda install mkl=2018
pip install pytorch-pretrained-bert==0.6.2

In [1]:
import re
import torch
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM

### Cargar el tokenizador BERT pre-entrenado y el paquete especializado en predecir una palabra ocultada (masked)

In [2]:
import time
start = time.time()
#Cargar el tokenizer pre-entrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Cargamos el modelo preparado para predecir una palabra ocultada (masked)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
end = time.time()
print ("Time Elapsed to load BERT ",end-start)

100%|██████████| 407873900/407873900 [00:16<00:00, 24951789.04B/s]


Time Elapsed to load BERT  25.242907524108887


### Predicción de las palabras que pueden ocupar un blank

In [3]:
#Función para predecir la 30 palabras que pueden llenar un blank según su probabilidad 
# Eg: The Sun is more ____ 4 billion years old.

def get_predicted_words(text):
    # Añadimos al texto los tokens de inicio [CLS] y token de separador (fin) [SEP]
    # Sustituimos '_______' por el token MASK
    text = "[CLS] " + text.replace("____", "[MASK]") + " [SEP]"
    # text= '[CLS] Tom has fully [MASK] from his illness. [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    # Mostrar los tokens de la frase según el tokenizador de BERT
    print("tokenized sentence: ",tokenized_text,"\n")
    # Obtener el índice del token '[MASK]'
    masked_index = tokenized_text.index('[MASK]')
    # Convertir los tokens a su índices según el vocabulario del modelo BERT 
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    print("tokenized sentence indexes: ",indexed_tokens,"\n")

    # Crear el tensor del segmento. Se inicia con un tensor de longitud del texto tokenizado con índices igual a 0 .
    segments_ids = [0] * len(tokenized_text)
    print("SEG IDS", segments_ids)

    # Convertir los inputs a tensores PyTorch
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predecir el token enmascarado según el modelo
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    # Obtener un número k de opciones candidatas a sustituir la palabra oculta. 
    k = 30
    predicted_index, predicted_index_values = torch.topk(predictions[0, masked_index], k)
    print(predicted_index)
    # Los índices de los tokens del vocabulario que son candidatos se transforman a tokens
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_index_values.tolist())
    # Lista de tokens que no contienen signos de puntuación
    filtered_tokens_to_remove_punctuation = []
    # Retornar las opciones que tienen signos de puntuación.
    for token in predicted_tokens:
        if re.match("^[a-zA-Z0-9_]*$", token):
            filtered_tokens_to_remove_punctuation.append(token)
        
    return filtered_tokens_to_remove_punctuation

In [4]:
sentence = "They all look tiny ____ they are so far away from the Earth."
print ("original sentence: ",sentence,"\n")
predicted_words = get_predicted_words(sentence)
print ("predicted choices: ", predicted_words)

original sentence:  They all look tiny ____ they are so far away from the Earth. 

tokenized sentence:  ['[CLS]', 'they', 'all', 'look', 'tiny', '[MASK]', 'they', 'are', 'so', 'far', 'away', 'from', 'the', 'earth', '.', '[SEP]'] 

tokenized sentence indexes:  [101, 2027, 2035, 2298, 4714, 103, 2027, 2024, 2061, 2521, 2185, 2013, 1996, 3011, 1012, 102] 

SEG IDS [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([10.5270,  9.8855,  9.8194,  9.4380,  8.4528,  8.0976,  8.0250,  7.7745,
         7.3686,  7.2621,  7.0900,  6.6434,  5.8249,  5.7217,  5.3661,  5.1552,
         5.0250,  4.8781,  4.7832,  4.5813,  4.5462,  4.4355,  4.3125,  4.2794,
         4.1162,  4.0410,  4.0337,  3.9101,  3.8251,  3.6499])
predicted choices:  ['because', 'and', 'but', 'as', 'since', 'like', 'when', 'considering', 'for', 'except', 'though', 'that', 'yet', 'although', 'while', 'now', 'if', 'where', 'here', 'even', 'so', 'after', 'from', 'given']


## Similitud entre frases con palabras polisémicas

### Instalar spacy-transformers

pip install spacy-transformers

### Obtener el modelo BERT

python -m spacy download en_trf_bertbaseuncased_lg

In [9]:
import spacy

#Cargar el modelo
nlp = spacy.load("en_trf_bertbaseuncased_lg")

#Procesar los textos a comparar según el modelo
text1 = nlp("Apple shares rose on the news.")
text2 = nlp("Apple sold fewer iPhones this quarter.")
text3 = nlp("Apple pie is delicious.")

print(text1, "vs", text2, ":", "Similarity: ", text1[0].similarity(text2[0]))
print(text1, "vs", text3, ":", "Similarity: ", text1[0].similarity(text3[0]))

Apple shares rose on the news. vs Apple sold fewer iPhones this quarter. : Similarity:  0.73428535
Apple shares rose on the news. vs Apple pie is delicious. : Similarity:  0.43365782
